In [29]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression , ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector


In [30]:
# fetch dataset 
concrete_compressive_strength = fetch_ucirepo(id=165) 

In [31]:
# data (as pandas dataframes) 
X = concrete_compressive_strength.data.features 
y = concrete_compressive_strength.data.targets 

In [32]:
X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=24,test_size=0.3)
elastic = ElasticNet()
elastic.fit(X_train,y_train)
y_pred = elastic.predict(X_test)
r2_score(y_test,y_pred)

0.5766806310401154

In [33]:
alphas = np.linspace(0.0001,10,20)
l1 = np.linspace(0.001,1,10)
scores = []
for a in alphas:
    for i in l1:
        elastic = ElasticNet(alpha=a,l1_ratio=i)
        elastic.fit(X_train,y_train)
        y_pred = elastic.predict(X_test)
        scores.append([a,i,r2_score(y_test,y_pred)])

df_scores = pd.DataFrame(scores,columns=['alpha','l1_ratio','score'])
df_scores.sort_values('score',ascending=False,inplace=True)
best_a = df_scores['alpha'].iloc[0]
best_l1 = df_scores['score'].iloc[0]
best_sc = df_scores['l1_ratio'].iloc[0]
print("Best Alpha ",best_a)
print("Best l1 ratio ",best_l1)
print("Best Score",best_sc)

Best Alpha  0.0001
Best l1 ratio  0.5771752511549486
Best Score 0.001


In [34]:
housing = pd.read_csv('Housing.csv')

In [35]:
housing.head()

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [40]:
X = housing.drop('price',axis=1)
y = housing['price']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3)
ohe = OneHotEncoder(sparse_output=False,drop='first').set_output(transform='pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude=object) ),
                            (ohe,make_column_selector(dtype_include=object) ),
                            verbose_feature_names_out = False).set_output(transform='pandas')
elastic = ElasticNet()
X_one_trn = ct.fit_transform(X_train)
X_one_tst = ct.transform(X_test)

In [39]:
alphas = np.linspace(0.0001,10,20)
l1 = np.linspace(0.001,1,10)
scores = []
for a in alphas:
    for i in l1:
        elastic = ElasticNet(alpha=a,l1_ratio=i)
        elastic.fit(X_train,y_train)
        y_pred = elastic.predict(X_one_tst)
        scores.append([a,i,r2_score(y_test,y_pred)])

df_scores = pd.DataFrame(scores,columns=['alpha','l1_ratio','score'])
df_scores.sort_values('score',ascending=False,inplace=True)
best_a = df_scores['alpha'].iloc[0]
best_l1 = df_scores['score'].iloc[0]
best_sc = df_scores['l1_ratio'].iloc[0]
print("Best Alpha ",best_a)
print("Best l1 ratio ",best_l1)
print("Best Score",best_sc)

ValueError: could not convert string to float: 'no'